# AES Encryption

Let's try to understand the fundamentals behind AES, and why it is the standard for encryption. The goal is to intuitively understand why it was chosen, and why it works, not just that it happens to work!

Questions:
- Why this specific algorithm, what other algorithms had weaknesses (ie xor)?
